<a href="https://colab.research.google.com/github/plantehenry/NeuralNetworksFinalProject/blob/main/ConvolutionTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100, CIFAR10
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms.functional import resize
from torchvision.transforms import CenterCrop
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Define Network

In [ ]:
class CNN(nn.Module):

    def __init__(self, numChannels):
        super(CNN, self).__init__()
        # try conv 1d 
        #back to 5 layers
        # Convolutional layers:
        self.conv1 = nn.Conv1d(in_channels = numChannels, out_channels = 25, kernel_size = 7, stride = 1)
        self.conv2 = nn.Conv1d(in_channels = 25, out_channels = 50, kernel_size = 5, stride = 1)
        self.conv3 = nn.Conv1d(in_channels = 50, out_channels = 75, kernel_size = 3, stride = 1)
        self.conv4 = nn.Conv1d(in_channels = 75, out_channels = 100, kernel_size = 3, stride = 1)
        self.conv5 = nn.Conv1d(in_channels = 100, out_channels = 150, kernel_size = 3, stride = 1)

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()

        self.maxpool = nn.MaxPool2d(kernel_size = (1,2), stride = (1,2))

        # Batch normalization layers:
        self.batchnorm1 = nn.BatchNorm2d(num_features = 30)
        self.batchnorm2 = nn.BatchNorm2d(num_features = 50)

        #BACK TO 2 OR 3 FULLY CONNECTED LAYER
        # Fully-connected layers:
        #ue sigmoid for all fully connecetd layers 
        # or tanh in first two and last one sigmoid
        self.fc1 = nn.Linear(in_features = 150, out_features= 75)
        self.fc2 = nn.Linear(in_features = 75, out_features= 50)
        self.fc3 = nn.Linear(in_features = 50, out_features= 3)

        #no dropout ot batchnorm initially
        # try with droput last
        self.dropout1 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout2 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout3 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout4 = nn.Dropout(p = args.dropoutRate)

    # Evaluation function
    def evaluate(self, model, dataloader, device):
        model.eval()
        running_loss = 0.0
        criterion = nn.MSELoss()  
        for data in dataloader:
                inputs, true = data
                inputs = inputs.to(device)
                true = true.to(device)
                outputs = model(inputs)

                loss = criterion(outputs, true)

                loss = loss.detach().cpu().numpy()
                running_loss += loss
        return(running_loss/ dataloader.__len__())


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        # x = self.dropout1(x)
        x = self.maxpool(x)
        # x = self.batchnorm1(x)
        x = self.conv2(x)
        x = self.relu(x)
        # x = self.dropout2(x)
        x = self.maxpool(x)
        # x = self.batchnorm2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # x = self.dropout3(x)
        x = self.conv4(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        x = self.conv5(x)
        x = self.relu(x)
        # x = self.maxpool(x)
        x = torch.flatten(x, 1)

        
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.tanh(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        # outmap_min, _ = torch.min(x, dim=1, keepdim=True)
        # outmap_max, _ = torch.max(x, dim=1, keepdim=True)
        # x = (x - outmap_min) / (outmap_max - outmap_min)


        return x

### Data Loaders

In [ ]:
import argparse
from torch.utils.data.dataloader import default_collate
import csv


class TrainData(torch.utils.data.Dataset):
    def __init__(self,args):
        self.args = args
        self.input_sequence , self.output_sequence = self.loadData()
    ## working on giving more than one sequence
    def loadData(self):
        # Read the text
        input_sequence = []
        with open(f"{self.args.workingDir}/{self.args.inputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
              new_row = []
              
              # wrap each value in row in an array
              for val in row:
                new_row.append([float(val)])

              # if this is the first row will need to initialize input sequence with arrays
              while len(new_row) > len(input_sequence):
                input_sequence.append([])
              
              # put each wrapped value in the correct index 
              for idx in range(len(new_row)):
                input_sequence[idx].append(new_row[idx])
        test_len = int(args.test_split * len(input_sequence[0]))
        input_sequence = torch.tensor(input_sequence)[:, 0:test_len, :]



        output_sequence = []
        with open(f"{self.args.workingDir}/{self.args.outputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
            new_row = []
            for val in row:
              new_row.append(float(val))
            output_sequence.append(new_row)
        output_sequence = torch.tensor(output_sequence)[0:test_len]
        return input_sequence, output_sequence

    def __len__(self):
        # Get the number of sequences for training purpose.
        return len(self.input_sequence[0]) - self.args.seqLength

    # returns tensor is size [num_features, 1, seq_length]
    def __getitem__(self, index):
        return_seq = self.input_sequence[:, index:index+self.args.seqLength, :].transpose(1, 2).detach().clone()
        return_seq = torch.squeeze(return_seq)
        # need to apply normalization to input 
        # normalization squeezes between 0 and 1 while keeping percent change between values the same
        # each feature sequence that I want to noralize gets divided by its max value

        # stock, bond, and real estate price
        # normalize_idxs = [0, 1]

        # # get mask with size of features with 1s where you want to normlaize
        # mask = torch.zeros(return_seq.shape[0], dtype=torch.bool)
        # mask[normalize_idxs] = True

        # # get max values of indees you want to normaize
        # max_vals, _ = return_seq[mask].max(dim = 2, keepdim =True)

        # # for indexes you want to nomalize divide by max value
        # return_seq[mask] /= max_vals
        return (
            return_seq,
            self.output_sequence[index+self.args.seqLength - 1],
        )


class ValData(torch.utils.data.Dataset):
    def __init__(self,args):
        self.args = args
        self.input_sequence , self.output_sequence = self.loadData()

    def loadData(self):
        # Read the text
        input_sequence = []
        with open(f"{self.args.workingDir}/{self.args.inputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
              new_row = []
              for val in row:
                new_row.append([float(val)])
              while len(new_row) > len(input_sequence):
                input_sequence.append([])
              for idx in range(len(new_row)):
                input_sequence[idx].append(new_row[idx])
        test_len = int(args.test_split * len(input_sequence[0]))
        input_sequence = torch.tensor(input_sequence)[:, test_len: , :]

        output_sequence = []
        with open(f"{self.args.workingDir}/{self.args.outputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
            new_row = []
            for val in row:
              new_row.append(float(val))
            output_sequence.append(new_row)
        output_sequence = torch.tensor(output_sequence)[test_len: ]
        return input_sequence, output_sequence

    def __len__(self):
        # Get the number of sequences for training purpose.
        return len(self.input_sequence[0]) - self.args.seqLength

    def __getitem__(self, index):
        return_seq = self.input_sequence[:, index:index+self.args.seqLength, :].transpose(1, 2).detach().clone()
        return_seq = torch.squeeze(return_seq)
        # need to apply normalization to input 
        # normalization squeezes between 0 and 1 while keeping percent change between values the same
        # each feature sequence that I want to noralize gets divided by its max value

        # # stock, bond, and real estate price
        # normalize_idxs = [0, 1]

        # # get mask with size of features with 1s where you want to normlaize
        # mask = torch.zeros(return_seq.shape[0], dtype=torch.bool)
        # mask[normalize_idxs] = True

        # # get max values of indees you want to normaize
        # max_vals, _ = return_seq[mask].max(dim = 2, keepdim =True)

        # # for indexes you want to nomalize divide by max value
        # return_seq[mask] /= max_vals

        return (
            return_seq,
            self.output_sequence[index+self.args.seqLength -1],
        )

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument('--workingDir', type=str, 
    default="/content/drive/My Drive/Neural Networks Project")
# Can be game_of_thrones or wonder_land:
parser.add_argument('--inputFile', type=str, default="input_data_conv") 
parser.add_argument('--outputFile', type=str, default="output_data_conv") 
parser.add_argument('--maxEpochs', type=int, default=100)
parser.add_argument('--batchSize', type=int, default=12)
parser.add_argument('--seqLength', type=int, default=63)
parser.add_argument('--learningRate', type=float, default=.01)
parser.add_argument('--dropoutRate', type=float, default=0.0)
parser.add_argument('--test_split', type=float, default=.9)
parser.add_argument('--num_channels', type=float, default=2)
args = parser.parse_args()

In [ ]:
dataset = TrainData(args)
train_loader = DataLoader(dataset,  batch_size=args.batchSize, shuffle=False, drop_last=False, num_workers=2)
valset = ValData(args)
val_loader = DataLoader(valset, batch_size=args.batchSize, shuffle=False, drop_last=False, num_workers=2)
data_iter = iter(train_loader)
print(dataset.__len__())
print(valset.__len__())

4222
414


### The main training and evaluation code starts here

In [ ]:
if __name__ == '__main__':
    # Specify the operation mode:
    # 'train' = training with your train and validation data splits
    # 'eval'  = evaluation of the trained model with your test data split 
    mode = 'train'

    # Path where you plan to save the best model during training
    my_best_model = "/content/drive/MyDrive/Neural Networks Project/Convolution_best_model_conv.pth"

    # Set the device (GPU or CPU, depending on availability)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Currently using device: ", device)

    # Initialize the model and print out its configuration
    model = CNN(numChannels = args.num_channels)
    model.to(device)
    print("\n\nModel summary:\n\n")
    summary(model, input_size=(args.num_channels, 63))

    if mode == "train":

        print("\n\nTraining starts!\n\n")
        
        model.train()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.learningRate)
        # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
        
        best_val_loss = float('inf')
        for epoch in range(args.maxEpochs):
            running_loss = .0
            print(f"Starting epoch {epoch + 1}")
            bestLoss = float('inf')
            for idx, data in tqdm(enumerate(train_loader), total=len(train_loader)):
                # Get the inputs (data is a list of [inputs, labels])
              
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                loss = loss.detach().cpu().numpy()
                inputs = inputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                running_loss += loss
              
            
                currLoss = loss.item()
                
                # Only save models with smallest loss per epoch.
                if currLoss < bestLoss:
                    bestLoss = currLoss
                    # torch.save(model.state_dict(), my_best_model)
            print(f"Epoch ID: {epoch}, 'the ave loss': {running_loss/ train_loader.__len__()}")

            # Evaluate the accuracy after each epoch
            val_loss = model.evaluate(model, val_loader, device)
            print(f"Epoch ID: {epoch}, 'the ave val loss': {val_loss}")
            if val_loss < best_val_loss:
                print(f"Better validation accuracy achieved: {val_loss}")
                best_val_loss = val_loss
                print(f"Saving this model as: {my_best_model}")
                torch.save(model.state_dict(), my_best_model)

    # And here we evaluate the trained model with the test data
    # elif mode == "eval":

    #     print("\n\nValidating the trained model:")
    #     print(f"Loading checkpoint from {my_best_model}")
    #     model.load_state_dict(torch.load(my_best_model))
    #     acc = model.evaluate(model, test_loader, classes, device)
    #     print(f"Accuracy on the test (unknown) data: {acc * 100:.2f}%")

    else:
        print("'mode' argument should either be 'train' or 'eval'")

Currently using device:  cuda:0


Model summary:


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 25, 57]             375
              ReLU-2               [-1, 25, 57]               0
         MaxPool2d-3               [-1, 25, 28]               0
            Conv1d-4               [-1, 50, 24]           6,300
              ReLU-5               [-1, 50, 24]               0
         MaxPool2d-6               [-1, 50, 12]               0
            Conv1d-7               [-1, 75, 10]          11,325
              ReLU-8               [-1, 75, 10]               0
         MaxPool2d-9                [-1, 75, 5]               0
           Conv1d-10               [-1, 100, 3]          22,600
             ReLU-11               [-1, 100, 3]               0
           Conv1d-12               [-1, 150, 1]          45,150
             ReLU-13               [-1, 150, 1]     

100%|██████████| 352/352 [00:03<00:00, 98.95it/s] 

Epoch ID: 0, 'the ave loss': 0.21395064783428097


Epoch ID: 0, 'the ave val loss': 0.20611051033650127
Better validation accuracy achieved: 0.20611051033650127
Saving this model as: /content/drive/MyDrive/Neural Networks Project/Convolution_best_model_conv.pth
Starting epoch 2


100%|██████████| 352/352 [00:02<00:00, 118.22it/s]

...

100%|██████████| 352/352 [00:02<00:00, 133.39it/s]

Epoch ID: 58, 'the ave loss': 0.21439874768111622


Epoch ID: 58, 'the ave val loss': 0.2040258641753878
Starting epoch 60


 61%|██████▏   | 216/352 [00:02<00:01, 107.89it/s]


KeyboardInterrupt: ignored

In [ ]:
import random
checkpointFile = "/content/drive/MyDrive/Neural Networks Project/Convolution_best_model_conv"

# If using GPU, we need the following line.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Currently using device: ", device)

# Initialize the model and print out its configuration
model = CNN(numChannels = args.num_channels)
model.to(device)

model.load_state_dict(torch.load(f"{checkpointFile}.pth"))


generatingLoader = DataLoader(valset, batch_size=1, shuffle=True)


# Get one data point from the DataLoader
data_point, true = next(iter(generatingLoader))
data_point = data_point.to(device)
# print(data_point)
print("true")
print(true)
model.eval()
torch.no_grad()
print("predicted")
print(model(data_point))

Currently using device:  cuda:0
true
tensor([[1.0000, 0.0000, 0.2899]])
predicted
tensor([[0.7650, 0.2131, 0.3131]], device='cuda:0', grad_fn=<SigmoidBackward0>)
